## Recuperation des images cropped

### Imports

In [14]:
import pandas as pd
import requests as req
import os.path
from datetime import datetime

https://skyserver.sdss.org/dr18/SkyServerWS/ImgCutout/getjpeg?TaskName=Skyserver.Chart.Navi&ra=5.433176&dec=12.065186&scale=0.099031675&radius=0.2&format=json

### CSV

In [15]:
df =pd.read_csv('../data/stars_classification_with_images.csv')

df = df[['obj_ID','alpha','delta', 'class']]

df

,obj_ID,alpha,delta,class
0,1.237680e+18,340.995121,20.589476,QSO
1,1.237679e+18,23.234926,11.418188,QSO
2,1.237679e+18,5.433176,12.065186,GALAXY
3,1.237661e+18,200.290475,47.199402,GALAXY
4,1.237671e+18,39.149691,28.102842,STAR
...,...,...,...,...
99995,1.237679e+18,39.620709,-2.594074,GALAXY
99996,1.237679e+18,29.493819,19.798874,GALAXY
99997,1.237668e+18,224.587407,15.700707,GALAXY
99998,1.237661e+18,212.268621,46.660365,GALAXY


### Clean Data

In [16]:
df['obj_ID'] = df['obj_ID'].apply(lambda x: int(x))

In [17]:
df

,obj_ID,alpha,delta,class
0,1237680272039609088,340.995121,20.589476,QSO
1,1237678858481565952,23.234926,11.418188,QSO
2,1237678858473963520,5.433176,12.065186,GALAXY
3,1237661435386659840,200.290475,47.199402,GALAXY
4,1237670961088167936,39.149691,28.102842,STAR
...,...,...,...,...
99995,1237678879964201472,39.620709,-2.594074,GALAXY
99996,1237679476408647936,29.493819,19.798874,GALAXY
99997,1237668298220634368,224.587407,15.700707,GALAXY
99998,1237661151383454208,212.268621,46.660365,GALAXY


### Get Cropped Images

In [2]:
#params = {
#    'ra': 5.433176,
#    'dec': 12.065186,
#    'scale': 0.099031675,
#    'radius': 0.2,
#    'format': 'json'
#}

In [19]:
images_location = '../../To-infinity-and-beyond-data/data/images_cropped'
logs = '../../To-infinity-and-beyond-data/data/images_cropped_log'

In [23]:
def append_to_log(text):
    timestamp = datetime.today().strftime('%Y-%m-%d')
    log_file = logs + '/' + timestamp + '.log'
    with open(log_file, "a") as myfile:
        myfile.write(text+'\n')

In [21]:

def cropped_image_seb(alpha, delta, object_ID):
    append_to_log(f'processed object_id {object_ID}')
    image_path = f'{images_location}/{object_ID}.jpeg'
    if not os.path.isfile(image_path):
        baseurl = 'https://skyserver.sdss.org/dr18/SkyServerWS/ImgCutout/getjpeg?TaskName=Skyserver.Chart.Navi&scale=0.099031675&radius=0.2&format=json'
        resp = req.get(baseurl, params={'ra': alpha, 'dec': delta})
        f = open(image_path,'wb')
        f.write(resp.content)
        f.close()
        append_to_log(f"downloaded {object_ID}")


In [26]:
for i in range(30001,30500):
    append_to_log(f"processed index {i}")
    cropped_image_seb(df.loc[i]['alpha'],df.loc[i]['delta'],df.loc[i]['obj_ID'])

### Upload to GCP

In [5]:
# https://cloud.google.com/storage/docs/samples/storage-upload-file?hl=en#storage_upload_file-python
from google.cloud import storage
images_location = '../../To-infinity-and-beyond-data/data/images_cropped'
object_ID = '1237646797600326400'
image_path = f'{images_location}/{object_ID}.jpeg'
client = storage.Client()
bucket = client.bucket('to-infinity-and-beyond')
blob = bucket.blob(f'{object_ID}.jpeg')
blob.upload_from_filename(image_path)

Forbidden: 403 POST https://storage.googleapis.com/upload/storage/v1/b/to-infinity-and-beyond/o?uploadType=multipart: {
  "error": {
    "code": 403,
    "message": "sebastien@taxifare1672-424708.iam.gserviceaccount.com does not have storage.objects.create access to the Google Cloud Storage object. Permission 'storage.objects.create' denied on resource (or it may not exist).",
    "errors": [
      {
        "message": "sebastien@taxifare1672-424708.iam.gserviceaccount.com does not have storage.objects.create access to the Google Cloud Storage object. Permission 'storage.objects.create' denied on resource (or it may not exist).",
        "domain": "global",
        "reason": "forbidden"
      }
    ]
  }
}
: ('Request failed with status code', 403, 'Expected one of', <HTTPStatus.OK: 200>)